In [1]:
import os
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import collections
from collections import Counter


In [2]:
def collect_metrics(root_dir):

    depth = 2

    dict_res_camem = dict()
    dict_res_xlm = dict()
    
    for subdir, dirs, files in os.walk(root_dir):
        if subdir[len(root_dir):].count(os.sep) < depth:
            for file_ in files:
                if file_ == "xai_metrics.json":
                    with open(subdir + "/" + file_) as f:
                        d = json.load(f)
                        log_odd_pos = d["log_odd_pos"]
                        anti_log_odd_pos = d["anti_log_odd_pos"]
                        comp_pos = d["comp_pos"]
                        suff_pos = d["suff_pos"]
                        log_odd_neg = d["log_odd_neg"]
                        anti_log_odd_neg = d["anti_log_odd_neg"]
                        comp_neg = d["comp_neg"]
                        suff_neg = d["suff_neg"]
                        
                    model_type = subdir.split(os.sep)[-1].split("_")[0]
                    if model_type == "camem":
                        dataset_name = " ".join(subdir.split(os.sep)[-1].split("_")[1:])
                        dict_res_camem[dataset_name] = {"log odd pos": log_odd_pos, "anti log odd pos": anti_log_odd_pos, "comp pos": comp_pos, "suff pos": suff_pos, "log odd neg": log_odd_neg, "anti log odd neg": anti_log_odd_neg, "comp neg": comp_neg, "suff neg": suff_neg}
                    elif model_type == "xlm":
                        dataset_name = " ".join(subdir.split(os.sep)[-1].split("_")[1:])
                        dict_res_xlm[dataset_name] = {"log odd pos": log_odd_pos, "anti log odd pos": anti_log_odd_pos, "comp pos": comp_pos, "suff pos": suff_pos, "log odd neg": log_odd_neg, "anti log odd neg": anti_log_odd_neg, "comp neg": comp_neg, "suff neg": suff_neg}

    return dict_res_camem, dict_res_xlm

def _average_dicts(dicts):
    avg_dict = collections.defaultdict(lambda: 0)
    
    for i, d in enumerate(dicts):
        for k, v in d.items():
            avg_dict[k] += v
        
    for k, v in avg_dict.items():
        avg_dict[k] /= (i+1)
    
    return avg_dict

def average_over_users(res):
    averaged_en = collections.defaultdict(list)
    averaged_it = collections.defaultdict(list)

    for k, v in res.items():
        if "en" in k and not ("nf" in k):
            averaged_en[" ".join(k.split(" ")[0:2])].append(v)
        elif "it" in k and not ("nf" in k):
            averaged_it[" ".join(k.split(" ")[0:2])].append(v)
    
    result_en = dict()
    result_it = dict()
            
    for k, v in averaged_en.items():
        result_en["avg en " + k] = _average_dicts(v)
        
    for k, v in averaged_it.items():
        result_it["avg it " + k] = _average_dicts(v)
                        
    return result_en, result_it

def process_to_present(res, column_width="1.1cm", plot_transpose=True):
    df_res = pd.DataFrame.from_dict(res).T
    # df_res = df_res.rename(columns=mapping_columns_names)
    if not plot_transpose:
        df_res = df_res.T
    s = df_res.style
    s.format(na_rep='MISS', precision=2)
    print(s.to_latex(column_format='l'+('p{'+f'{column_width}'+'}')*len(df_res.columns)))

In [3]:
dict_res_camem, dict_res_xlm = collect_metrics("../output/sentipolc")

## CAMEM

In [4]:
_, avg = average_over_users(dict_res_camem)
camem_results = {**dict_res_camem, ** avg}

In [5]:
pd.DataFrame(camem_results).T

,log odd pos,anti log odd pos,comp pos,suff pos,log odd neg,anti log odd neg,comp neg,suff neg
np nf it,-0.691565,-0.543487,0.381319,0.062397,-2.214154,-3.404119,0.366116,0.013536
np f it1,-0.499680,-0.495662,0.138060,0.097855,-0.486616,-1.008763,0.121911,0.255329
np f it26,-0.454215,-0.449846,0.149973,0.136133,-0.455345,-0.922343,0.126213,0.185089
np f it44,-0.520989,-0.611196,0.120288,0.102873,-0.656536,-0.862651,0.164041,0.168058
np f it43,-0.436473,-0.672271,0.116478,0.154069,-0.414820,-0.781545,0.125521,0.183521
p f it38,-0.229622,-1.685469,0.054186,0.336091,-0.477379,-1.063864,0.126802,0.171620
p f it43,-0.549073,-0.571548,0.132629,0.175801,-0.368780,-0.831373,0.116129,0.155114
p f it44,-0.478448,-0.468194,0.129998,0.149744,-0.658347,-0.608002,0.183118,0.178377
p f it26,-0.586587,-0.467296,0.142025,0.127413,-0.679425,-1.013140,0.171474,0.230134
np f it38,-0.568351,-0.698348,0.112129,0.124738,-0.554102,-0.965582,0.126461,0.199545


In [6]:
process_to_present(camem_results)

\begin{tabular}{lp{1.1cm}p{1.1cm}p{1.1cm}p{1.1cm}p{1.1cm}p{1.1cm}p{1.1cm}p{1.1cm}}
 & log odd pos & anti log odd pos & comp pos & suff pos & log odd neg & anti log odd neg & comp neg & suff neg \\
np nf it & -0.69 & -0.54 & 0.38 & 0.06 & -2.21 & -3.40 & 0.37 & 0.01 \\
np f it1 & -0.50 & -0.50 & 0.14 & 0.10 & -0.49 & -1.01 & 0.12 & 0.26 \\
np f it26 & -0.45 & -0.45 & 0.15 & 0.14 & -0.46 & -0.92 & 0.13 & 0.19 \\
np f it44 & -0.52 & -0.61 & 0.12 & 0.10 & -0.66 & -0.86 & 0.16 & 0.17 \\
np f it43 & -0.44 & -0.67 & 0.12 & 0.15 & -0.41 & -0.78 & 0.13 & 0.18 \\
p f it38 & -0.23 & -1.69 & 0.05 & 0.34 & -0.48 & -1.06 & 0.13 & 0.17 \\
p f it43 & -0.55 & -0.57 & 0.13 & 0.18 & -0.37 & -0.83 & 0.12 & 0.16 \\
p f it44 & -0.48 & -0.47 & 0.13 & 0.15 & -0.66 & -0.61 & 0.18 & 0.18 \\
p f it26 & -0.59 & -0.47 & 0.14 & 0.13 & -0.68 & -1.01 & 0.17 & 0.23 \\
np f it38 & -0.57 & -0.70 & 0.11 & 0.12 & -0.55 & -0.97 & 0.13 & 0.20 \\
p f it1 & -0.51 & -0.36 & 0.11 & 0.13 & -0.57 & -0.73 & 0.18 & 0.19 \\
p nf it 

## XLM

In [7]:
_, avg_it = average_over_users(dict_res_xlm)
xlm_results = {**dict_res_xlm, **avg_it}

In [8]:
pd.DataFrame(xlm_results).T

,log odd pos,anti log odd pos,comp pos,suff pos,log odd neg,anti log odd neg,comp neg,suff neg
np f it1,-0.116605,-0.464706,0.037889,0.182034,-0.034269,-0.032010,0.002224,0.131151
np nf it,-2.015370,-3.081086,0.427266,-0.006541,-1.071291,-0.166928,0.367231,0.008050
p f it38,-0.153110,-1.085922,0.030529,0.283109,-0.036450,-0.032415,0.003442,0.063298
np f it38,-0.005451,-0.121301,0.009724,0.045558,-0.023759,-0.006580,0.008489,0.042604
np f it26,-0.009730,-0.600356,0.006545,0.518024,-0.004353,-0.033423,0.000382,0.021022
np f it44,-0.027169,-0.319342,0.007843,0.140587,-0.017884,0.014445,0.011308,-0.003676
np f it43,-0.055469,-0.105202,0.021579,0.099746,-0.103805,-0.110801,0.036014,0.096221
p f it1,-0.044921,-0.058307,0.008640,0.069578,-0.511053,-0.435316,0.143204,0.120201
p f it26,-0.107836,-0.376501,0.001506,0.170017,-0.082953,-0.039990,0.012924,0.115964
p f it44,-0.074108,-0.044138,0.009702,-0.010294,0.002363,-0.033619,0.004774,0.294184


In [9]:
process_to_present(xlm_results)

\begin{tabular}{lp{1.1cm}p{1.1cm}p{1.1cm}p{1.1cm}p{1.1cm}p{1.1cm}p{1.1cm}p{1.1cm}}
 & log odd pos & anti log odd pos & comp pos & suff pos & log odd neg & anti log odd neg & comp neg & suff neg \\
np f it1 & -0.12 & -0.46 & 0.04 & 0.18 & -0.03 & -0.03 & 0.00 & 0.13 \\
np nf it & -2.02 & -3.08 & 0.43 & -0.01 & -1.07 & -0.17 & 0.37 & 0.01 \\
p f it38 & -0.15 & -1.09 & 0.03 & 0.28 & -0.04 & -0.03 & 0.00 & 0.06 \\
np f it38 & -0.01 & -0.12 & 0.01 & 0.05 & -0.02 & -0.01 & 0.01 & 0.04 \\
np f it26 & -0.01 & -0.60 & 0.01 & 0.52 & -0.00 & -0.03 & 0.00 & 0.02 \\
np f it44 & -0.03 & -0.32 & 0.01 & 0.14 & -0.02 & 0.01 & 0.01 & -0.00 \\
np f it43 & -0.06 & -0.11 & 0.02 & 0.10 & -0.10 & -0.11 & 0.04 & 0.10 \\
p f it1 & -0.04 & -0.06 & 0.01 & 0.07 & -0.51 & -0.44 & 0.14 & 0.12 \\
p f it26 & -0.11 & -0.38 & 0.00 & 0.17 & -0.08 & -0.04 & 0.01 & 0.12 \\
p f it44 & -0.07 & -0.04 & 0.01 & -0.01 & 0.00 & -0.03 & 0.00 & 0.29 \\
p f it43 & -0.06 & -0.16 & 0.03 & 0.28 & -0.10 & -0.07 & 0.04 & 0.08 \\
p nf it